<a href="https://colab.research.google.com/github/yusuku/DepthEstimationFrom360/blob/main/RectNetForDepthEstiamtion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [12]:
input_shape = (224, 224, 3)
input_tensor = tf.keras.layers.Input(shape=input_shape)

# Apply ELU activation within the Conv2D layers
input0_0_out = tf.keras.layers.Conv2D(8, (3, 9), activation='elu')(input_tensor)
input0_1_out = tf.keras.layers.Conv2D(8, (5,11), activation='elu')(input_tensor)
input0_2_out = tf.keras.layers.Conv2D(8, (5, 7), activation='elu')(input_tensor)
input0_3_out = tf.keras.layers.Conv2D(8, (7, 7), activation='elu')(input_tensor)

# Concatenate the outputs
input0_out_cat=tf.keras.layers.Concatenate()([input0_0_out,input0_1_out,input0_2_out,input0_3_out])

input1_0_out = tf.keras.layers.Conv2D(16, (3,9), activation='elu')(input0_out_cat)
input1_1_out = tf.keras.layers.Conv2D(16, (3,7), activation='elu')(input0_out_cat)
input1_2_out = tf.keras.layers.Conv2D(16, (3,5), activation='elu')(input0_out_cat)
input1_3_out = tf.keras.layers.Conv2D(16, (5,5), activation='elu')(input0_out_cat)

input1_out_cat=tf.keras.layers.Concatenate()([input0_0_out,input0_1_out,input0_2_out,input0_3_out])

encoder0_0_out = tf.keras.layers.Conv2D(128, (3,3),strides=2, activation='elu')(input1_out_cat)
encoder0_1_out = tf.keras.layers.Conv2D(128, (3,3), activation='elu')(encoder0_0_out)
encoder0_2_out = tf.keras.layers.Conv2D(128, (3,3), activation='elu')(encoder0_1_out)

encoder1_0_out = tf.keras.layers.Conv2D(256, (3,3),strides=2, activation='elu')(encoder0_2_out)
encoder1_1_out = tf.keras.layers.Conv2D(256, (3,3),strides=1, activation='elu')(encoder1_0_out)
encoder1_2_out = tf.keras.layers.Conv2D(256, (3,3),strides=1, activation='elu')(encoder1_1_out)
encoder1_3_out = tf.keras.layers.Conv2D(256, (3,3),strides=1, activation='elu')(encoder1_1_out)

encoder1_out_cat=tf.keras.layers.Concatenate()([encoder1_1_out,encoder1_3_out])


encoder2_0_out = tf.keras.layers.Conv2D(512, (3,3),strides=1,dilation_rate=6 ,activation='elu')(encoder1_out_cat)
encoder2_1_out = tf.keras.layers.Conv2D(512, (3,3),strides=1,dilation_rate=16 , activation='elu')(encoder2_0_out)
encoder2_2_out = tf.keras.layers.Conv2D(512, (1,1),strides=1, activation='elu',padding='same')(encoder2_1_out)

encoder2_out_cat=tf.keras.layers.Concatenate()([encoder2_0_out,encoder2_2_out])

decoder0_0_out = tf.keras.layers.Conv2DTranspose(256, (4,4),strides=2, activation='elu')(encoder2_out_cat)
decoder0_1_out = tf.keras.layers.Conv2D(256, (5,5),strides=1 ,activation='elu')(decoder0_0_out)



decoder1_0_out = tf.keras.layers.Conv2DTranspose(128, (4,4),strides=2, activation='elu')(decoder0_1_out)
decoder1_1_out = tf.keras.layers.Conv2D(128, (5,5),strides=1 ,activation='elu')(decoder1_0_out)

dropout_decoder0_1_out=tf.keras.layers.Dropout(0.05)(decoder0_1_out)
dropout_decoder0_1_out=tf.keras.layers.Conv2D(128, (3,3),strides=1 )(decoder1_0_out)

decoder1_out_cat=tf.keras.layers.Concatenate()([dropout_decoder0_1_out,decoder1_1_out])

decoder2_0_out =tf.keras.layers.Conv2D(64, (1,1),strides=1, activation='elu')(decoder1_out_cat)


output=tf.keras.layers.Dropout(0.08)(decoder2_0_out)
output=tf.keras.layers.Conv2D(1, (1,1),strides=1)(output)

model = tf.keras.Model(inputs=input_tensor, outputs=output)
model.summary()


ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 222, 216, 8), (None, 220, 214, 8), (None, 220, 218, 8), (None, 218, 218, 8)]